# Test

## Test load model

### VLT5

In [2]:
from cvlep.trainer_base import Trainer

config_encoder_question = "experiments/configEncoder/args_VLT5.json"
config_encoder_passage = "experiments/configEncoder/args_VLT5.json"
config_training = "experiments/configEncoder/training_params/training_vlt5.json"

trainer = Trainer(config_encoder_question, config_encoder_passage, config_training)

Some weights of the model checkpoint at data_model/t5_pretrained were not used when initializing JointEncoder: ['shared.weight', 'encoder.embed_tokens.weight', 'encoder.block.0.layer.0.SelfAttention.q.weight', 'encoder.block.0.layer.0.SelfAttention.k.weight', 'encoder.block.0.layer.0.SelfAttention.v.weight', 'encoder.block.0.layer.0.SelfAttention.o.weight', 'encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.0.layer.0.layer_norm.weight', 'encoder.block.0.layer.1.DenseReluDense.wi.weight', 'encoder.block.0.layer.1.DenseReluDense.wo.weight', 'encoder.block.0.layer.1.layer_norm.weight', 'encoder.block.1.layer.0.SelfAttention.q.weight', 'encoder.block.1.layer.0.SelfAttention.k.weight', 'encoder.block.1.layer.0.SelfAttention.v.weight', 'encoder.block.1.layer.0.SelfAttention.o.weight', 'encoder.block.1.layer.0.layer_norm.weight', 'encoder.block.1.layer.1.DenseReluDense.wi.weight', 'encoder.block.1.layer.1.DenseReluDense.wo.weight', 'encoder.block.1.layer.1.

AttributeError: 'NoneType' object has no attribute 'weight'

In [17]:
token = trainer.tokenizer_passage('lapin [SEP]').input_ids

In [18]:
trainer.tokenizer_passage.decode(token)

'lapin [SEP]</s>'

In [2]:
from cvlep.VLT5.tokenization import VLT5Tokenizer, VLT5TokenizerFast
tokenizer = VLT5TokenizerFast.from_pretrained('data/tokenizer/t5-base')

In [23]:
from transformers import T5Config
config_class = T5Config
config = config_class.from_pretrained("data_model/t5_pretrained")

import json

with open("experiments/configEncoder/args_VLT5.json", 'r') as f:
    args = json.load(f)

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict(args)
config.feat_dim = args.feat_dim
config.pos_dim = args.pos_dim
config.n_images = 2

config.use_vis_order_embedding = args.use_vis_order_embedding

config.dropout_rate = args.dropout
config.dropout = args.dropout
config.attention_dropout = args.dropout
config.activation_dropout = args.dropout

config.use_vis_layer_norm = args.use_vis_layer_norm
config.individual_vis_layer_norm = args.individual_vis_layer_norm
config.losses = args.losses

config.share_vis_lang_layer_norm = args.share_vis_lang_layer_norm
config.classifier = args.classifier

from cvlep.VLT5.modeling_t5 import JointEncoder

model_name = "data_model/t5_pretrained"
model = JointEncoder.from_pretrained(
    model_name,
    config=config
)

In [24]:
# charger checkpoint via state dict
from cvlep.VLT5.utils import load_state_dict
ckpt_path = "data_model/jointEncoder/state_dict/VLT5epoch30"
state_dict = load_state_dict(ckpt_path, 'cpu')
original_keys = list(state_dict.keys())

In [25]:
original_keys = list(state_dict.keys())
for key in original_keys:
    if key.startswith("vis_encoder."):
        new_key = 'encoder.' + key[len("vis_encoder."):]
        state_dict[new_key] = state_dict.pop(key)

    if key.startswith("model.vis_encoder."):
        new_key = 'model.encoder.' + key[len("model.vis_encoder."):]
        state_dict[new_key] = state_dict.pop(key)

results = model.load_state_dict(state_dict, strict=False)

JointEncoder(
  (visual_embedding): VisualEmbedding(
    (feat_embedding): Sequential(
      (0): Linear(in_features=2048, out_features=768, bias=True)
      (1): T5LayerNorm()
    )
    (absolute_vis_pos_embedding): Sequential(
      (0): Linear(in_features=5, out_features=768, bias=True)
      (1): T5LayerNorm()
    )
    (img_order_embedding): Embedding(2, 768)
  )
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=768, out_features=768, bias=False)
            (k): Linear(in_features=768, out_features=768, bias=False)
            (v): Linear(in_features=768, out_features=768, bias=False)
            (o): Linear(in_features=768, out_features=768, bias=False)
            (relative_attention_bias): Embedding(32, 12)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerFF(
          

### VL BART

In [8]:
from cvlep.trainer_base import Trainer

path_config = "experiments/model_cvlep/encodersBart.json"
trainer = Trainer(path_config)

In [9]:
trainer.model.config.config_str

"{'backbone_passage': 'bart',\n 'backbone_question': 'bart',\n 'embedding_dim': 768,\n 'num_embeddings': 50465,\n 'shared_embedding': True,\n 'use_projection': False}"

In [5]:
# just try to load without apply embedding and see if we already have the embedding
from transformers import BartConfig
from cvlep.VLT5.modeling_bart import JointEncoder
temp = BartConfig.from_pretrained('data_model/jointEncoder/Bartepoch30')
encoder = JointEncoder(temp)
        

In [6]:
from torch import nn
import torch
from cvlep.utils import device
embedding = nn.Embedding(50465,768)
embedding.load_state_dict(torch.load("data_model/embedding/Bartepoch30", map_location=device))

<All keys matched successfully>

In [7]:
embedding.weight

Parameter containing:
tensor([[ 0.0087,  0.0107, -0.0100,  ...,  0.0134,  0.1015,  0.0041],
        [ 0.0052,  0.0013, -0.0265,  ..., -0.0048,  0.0138, -0.0064],
        [ 0.0361,  0.0057,  0.0085,  ...,  0.0547,  0.0270,  0.0469],
        ...,
        [ 0.0291, -0.0131, -0.0512,  ..., -0.0230, -0.0292,  0.0559],
        [ 0.0612, -0.0263, -0.0298,  ..., -0.0385, -0.0123,  0.0363],
        [ 0.0481,  0.0120, -0.0077,  ..., -0.0404, -0.0105,  0.0497]],
       requires_grad=True)

In [4]:
trainer.model.image_question_encoder.embed_tokens.weight

Parameter containing:
tensor([[ 0.0082, -0.0147,  0.0142,  ..., -0.0429,  0.0094, -0.0031],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0161,  0.0231,  0.0134,  ..., -0.0171,  0.0082,  0.0079],
        ...,
        [ 0.0042,  0.0283,  0.0208,  ...,  0.0025,  0.0201, -0.0083],
        [ 0.0136,  0.0103,  0.0255,  ..., -0.0037,  0.0096, -0.0101],
        [ 0.0113, -0.0148, -0.0266,  ..., -0.0179,  0.0142, -0.0216]],
       requires_grad=True)

In [28]:
encoder.embed_tokens

Embedding(50465, 768, padding_idx=1)

## Tokenizer

In [24]:
from transformers import T5TokenizerFast


In [25]:
tokenizer = T5TokenizerFast.from_pretrained("t5-small")


In [30]:
tokenizer('Michelle Obama. She stand for')

{'input_ids': [15275, 4534, 5, 451, 1518, 21, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [33]:
tokenizer.decode([5])

'.'

In [13]:
token = tokenizer("James Bond [SEP]").input_ids


In [4]:
tokenizer("pikachu")


{'input_ids': [2816, 1258, 8019, 1], 'attention_mask': [1, 1, 1, 1]}

In [12]:
tokenizer.decode(token)


'James Bond [SEP]</s>'

In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [22]:
tokenizer("James bond [SEP] bond")


{'input_ids': [101, 2508, 5416, 102, 5416, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [23]:
tokenizer('James Bond')

{'input_ids': [101, 2508, 5416, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

## Test de sortie T5

In [17]:
from transformers import T5EncoderModel, T5TokenizerFast


In [18]:
encoder = T5EncoderModel.from_pretrained("t5-small")
tokenizer = T5TokenizerFast.from_pretrained('t5-small')


Some weights of the model checkpoint at t5-small were not used when initializing T5EncoderModel: ['decoder.block.1.layer.2.DenseReluDense.wo.weight', 'decoder.block.1.layer.0.SelfAttention.v.weight', 'decoder.block.3.layer.2.DenseReluDense.wo.weight', 'decoder.block.1.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.1.EncDecAttention.v.weight', 'decoder.block.2.layer.1.EncDecAttention.q.weight', 'decoder.block.5.layer.1.EncDecAttention.v.weight', 'decoder.block.2.layer.1.EncDecAttention.v.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.4.layer.1.layer_norm.weight', 'decoder.block.3.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.5.layer.0.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.0.layer_norm.weight', 'decoder.block.1.layer.1.EncDecAttention.k.weight', 'decoder.block.5.layer.0.SelfAttention.o.weight

In [19]:
input_ids = tokenizer(
    [
        'summarize: the situation is complicated but not dangerous',
        'work work'
        ], 
    return_tensors='pt', padding = True)
print(input_ids)

B, L = input_ids['input_ids'].size()[:-1]


{'input_ids': tensor([[21603,    10,     8,  1419,    19,  6446,    68,    59,  5107,     1],
        [  161,   161,     1,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}


ValueError: not enough values to unpack (expected 2, got 1)

In [31]:
output = encoder(output_hidden_states=True, **input_ids)


In [32]:
output.last_hidden_state.shape


torch.Size([1, 10, 512])

In [33]:
last_all_hidden_states = output[0]
all_hidden_states = output[1]


In [34]:
print("dim last hidden :", last_hidden_states.shape)
print(len(all_hidden_states), ' layers * 7x512')


dim last hidden : torch.Size([1, 7, 512])
7  layers * 7x512


In [35]:
print("Number of layers:", len(all_hidden_states),
      "  (initial embeddings + X layers)")
layer_i = 0

print("Number of batches:", len(all_hidden_states[layer_i]))
batch_i = 0

print("Number of tokens:", len(all_hidden_states[layer_i][batch_i]))
token_i = 0

print("Number of hidden units:", len(
    all_hidden_states[layer_i][batch_i][token_i]))


Number of layers: 7   (initial embeddings + X layers)
Number of batches: 1
Number of tokens: 10
Number of hidden units: 512


0 : Embedding


In [54]:
all_hidden_states[layer_i].shape

torch.Size([1, 10, 512])

In [ ]:
encoder

## Test pooler output CLIP

In [3]:
from transformers import CLIPTextModel, CLIPTokenizer

In [4]:
model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.5.layer_norm1.bias', 'vision_model.encoder.layers.2.self_attn.out_proj.weight', 'vision_model.encoder.layers.0.self_attn.k_proj.weight', 'vision_model.encoder.layers.4.mlp.fc2.weight', 'vision_model.encoder.layers.3.self_attn.out_proj.bias', 'vision_model.encoder.layers.10.mlp.fc2.weight', 'vision_model.encoder.layers.5.mlp.fc2.weight', 'vision_model.encoder.layers.8.layer_norm1.weight', 'vision_model.encoder.layers.8.layer_norm2.weight', 'vision_model.encoder.layers.8.self_attn.k_proj.weight', 'vision_model.encoder.layers.3.mlp.fc1.weight', 'vision_model.post_layernorm.bias', 'vision_model.encoder.layers.9.layer_norm1.bias', 'vision_model.encoder.layers.7.mlp.fc1.bias', 'vision_model.encoder.layers.6.self_attn.q_proj.weight', 'vision_model.encoder.layers.0.self_attn.q_proj.weight', 'vision_model.encoder.layers.10.layer_norm1.weight', 'vision

In [10]:
inputs = tokenizer(["a photo of a cat"], padding=True, return_tensors="pt")

In [11]:
outputs = model(**inputs)

last_hidden_state = outputs.last_hidden_state

pooled_output = outputs.pooler_output

print('dim hidden :', last_hidden_state.shape)
print('dim pool : ',pooled_output.shape)

dim hidden : torch.Size([1, 7, 512])
dim pool :  torch.Size([1, 512])


In [7]:
pooled_output.shape

torch.Size([2, 512])

## Test chargement des encoders

In [2]:
from cvlep.VLT5.modeling_t5 import JointEncoder as t5encoder
from cvlep.VLT5.modeling_bart import JointEncoder as BartEncoder

In [3]:
PATH = 'data/jointEncoder/'
encoder1 = BartEncoder.from_pretrained(f'{PATH}/bart_without_pretrain')
encoder2 = BartEncoder.from_pretrained(f'{PATH}/Bartepoch30')
encoder3 = t5encoder.from_pretrained(f'{PATH}/VLT5_without_pretrain')
encoder4 = t5encoder.from_pretrained(f'{PATH}/VLT5epoch30')

# OK it worsks

Some weights of the model checkpoint at data/jointEncoder//VLT5_without_pretrain were not used when initializing JointEncoder: ['embed_tokens.weight', 'visual_embedding.obj_order_embedding.weight']
- This IS expected if you are initializing JointEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing JointEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at data/jointEncoder//VLT5epoch30 were not used when initializing JointEncoder: ['embed_tokens.weight', 'visual_embedding.obj_order_embedding.weight']
- This IS expected if you are initializing JointEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializ

In [4]:
encoder1.config
# on doit pouvoir changer la T5 config ici pour en faire ce qu'on veut

BartConfig {
  "_name_or_path": "data/jointEncoder//bart_without_pretrain",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "JointEncoder"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier": false,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "default_obj_order_ids": [
    50464,
    50463,
    50462,
    50461,
    50460,
    50459,
    50458,
    50457,
    50456,
    50455,
    50454,
    50453,
    50452,
    50451,
    50450,
    50449,
    50448,
    50447,
    50446,
    50445,
    50444,
    50443,
    50442,
    50441,
    50440,
    50439,
    50438,
    50437,
    50436,
    50435,
    50434,
    50433,
    50432,
    50431,
    50430,
    50429,
    50428,
    50427,
    50426,
    50425,


## Test de Config

In [1]:
from cvlep.VLT5.param import Config

path = 'experiments/model_cvlep/encodersT5.json'
cfg, k = Config.load_json(path)

/home/pgrimal/miniconda3/envs/cvlep/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'data/jointEncoder/VLT5epoch30'

In [9]:
for a, v in k.items():
    print(a)
    if isinstance(v, dict):
        print(len(v), v)

encoder_question
2 {'model': {'backbone': 't5', 'pretrain': True, 'pretrained_model_name_or_path': 'data/jointEncoder/VLT5epoch30'}, 'tokenizer': {'backbone': 't5', 'use_vision': True, 'pretrained_model_name_or_path': 'data/t5_pretrained', 'max_length': 100, 'do_lower_case': True}}
encoder_passage
2 {'model': {'backbone': 't5', 'pretrain': True, 'pretrained_model_name_or_path': 'data/jointEncoder/VLT5epoch30'}, 'tokenizer': {'backbone': 't5', 'use_vision': True, 'pretrained_model_name_or_path': 'data/t5_pretrained', 'max_length': 100, 'do_lower_case': True}}
cvlep
1 {'use_projection': False}


## Test charger tokenizer

In [1]:
from transformers import BartTokenizer, T5Tokenizer

/home/pgrimal/miniconda3/envs/cvlep/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
bart_tokenizer.save_pretrained('data/tokenizer/bart-base')

('data/tokenizer/bart-base/tokenizer_config.json',
 'data/tokenizer/bart-base/special_tokens_map.json',
 'data/tokenizer/bart-base/vocab.json',
 'data/tokenizer/bart-base/merges.txt',
 'data/tokenizer/bart-base/added_tokens.json')

In [2]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_tokenizer.save_pretrained('data/tokenizer/t5-base')


('data/tokenizer/t5-base/tokenizer_config.json',
 'data/tokenizer/t5-base/special_tokens_map.json',
 'data/tokenizer/t5-base/spiece.model',
 'data/tokenizer/t5-base/added_tokens.json')